In [2]:
import warnings
import numpy as np
import os, re

from tqdm import tqdm
from glob import glob
from commons import plot
from skimage import img_as_ubyte
from skimage.io import imsave, imread_collection
from skimage.measure import regionprops

warnings.filterwarnings('ignore')

In [3]:
def find_bighest_cluster_area(clusters):
    
    regions = regionprops(clusters)

    all_areas, *_ = list(map(lambda item: item.area, regions))

    return all_areas

def local_median(arr):
    
    arr_sorted = sorted(arr)
    size = len(arr)
    index = 1
    
    if size == 1: return arr_sorted[size -1] # [2] -> 2
    if size == 2: return arr_sorted[size -1] # [2, 1] => [1, 2] => 2

    return arr_sorted[size // 2] if size >= 3 else index # [2, 1, 3] => [1, 2, 3] => 2


def find_median_image(path):
    images = imread_collection(path)

    broilers_frames = list(map(find_bighest_cluster_area, images))
    
    index_median_image = local_median(broilers_frames)

    images_indexes = dict(zip(broilers_frames, images))
    
    return images_indexes[int(index_median_image)]

In [4]:
def fames_processing(destiny_images, index):
    
    folder_number, *_ = re.findall(r'\d+', index)

    frame = find_median_image(f'{index}/*')
    
    imsave(f'{destiny_images}/{folder_number}.png', img_as_ubyte(frame))

In [5]:
try:
    destiny_images = 'outputs/selecteds_unique'
    os.mkdir(destiny_images)
except: print("Pasta já existe!")

In [6]:
for index in tqdm(glob("outputs/selecteds/*")): 
    fames_processing(destiny_images, index)

100%|██████████████████████████████████████████████████████| 161/161 [00:02<00:00, 65.41it/s]
